## Project-4 Подготовка рекомендательной таблицы учебных курсов. 
#### Автор: Сергей Нечаев. 
#### SkillFactory: SDA-3.0 Полный курс по анализу данных.


### ЦЕЛЬ
#### Подготовить основу рекомендательной системы.


### ЗАДАЧИ
Продакт-менеджер онлайн-школы MasterMind ожидает получить от вас рекомендательную систему, благодаря которой можно будет предлагать клиентам интересные им курсы и тем самым повышать средний чек.  
Вы решаете, что изначальным воплощением этой системы может стать таблица, в которой курсам будет соответствовать по две рекомендации.   
Кроме того, вы планируете вместе с отчётом (таблицей рекомендаций) скинуть продакту ещё и все написанные в процессе скрипты, чтобы было меньше вопросов по решению :)    
Ну, и раз в код будут смотреть не только ваши глаза, вы считаете необходимым снабдить его комментариями, которые бы разъясняли, что где и почему вы делаете.    
Также вы понимаете, что перед внедрением фичи коллеги решат провести A/B-тест и вас скорее всего привлекут к анализу результатов.   
Перспективы ясны, можно переходить к формализации задач.

### КОНКРЕТНЫЕ ШАГИ (ФОРМАЛИЗОВАННЫЕ ЗАДАЧИ)

1. Познакомиться с датасетом. Подготовить и проанализировать данные с помощью SQL.
2. Обработать данные средствами Python.
3. Составить итоговую таблицу с рекомендациями, снабдив её необходимыми комментариями, и представить отчёт продакт-менеджеру.
4. Проанализировать результаты A/B-теста, проведённого после внедрения фичи, и сделать вывод.



### ИСХОДНЫЕ ДАННЫЕ

#### Таблица carts — данные о пользовательских корзинах
Promo Code ID — ID промокода, если он есть   
Purchased At — дата оплаты    
User ID — ID пользователя   
Created At — дата создания корзины   
Updated At — дата последнего обновления информации   
ID — идентификатор корзины   
State — состояние оплаты  
#### Таблица cart items — данные о курсах, которые пользователи добавили в корзину
Created At — дата создания события   
Resource Type — тип продукта   
Resource ID — ID курса   
Cart ID — идентификатор корзины   
Updated At — дата последнего обновления информации   
ID — идентификатор операции  

### ЭТАПЫ ВЫПОЛНЕНИЯ РАБОТЫ

#### 1. ИЗУЧЕНИЕ И ПОДГОТОВКА ДАННЫХ ДЛЯ ДАЛЬНЕЙШЕГО АНАЛИЗА С ПОМОЩЬЮ SQL

In [26]:
 ! pip install psycopg2 


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
%matplotlib inline
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras 

In [28]:
# с помощью SQL проверим, за какой период имеются данные

def getDataPeriod():
    query = '''select 
    min(purchased_at) min_date,
    max(purchased_at) max_date
    from final.carts
    where state = 'successful'
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
data_period= pd.DataFrame(getDataPeriod())
data_period

,min_date,max_date
0,2017-01-01 07:29:15.164,2018-12-30 23:29:38.892


In [29]:
# С помощью SQL запроса смотрим, сколько всего имеется курсов

def getCourseAll():
    query = '''select 
    distinct ci.resource_id course_id
    from final.cart_items ci
    where ci.resource_type = 'Course'
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
course_id= pd.DataFrame(getCourseAll())

print(course_id.head(), '\n', 'всего курсов:', len(course_id))

   course_id
0        809
1       1145
2        908
3        814
4       1181 
 всего курсов: 127


In [30]:
# В SQL запросе смотрим, сколько клиентов купили курсы и сколько в среднем на одного клиента покупок курсов

def getUsersCourse():
    query = '''with purchased as
    (select 
    c.user_id user_id,
    count(ci.resource_id) count_course
    from
    final.carts c
    join final.cart_items ci on c.id = ci.cart_id
    where c.state = 'successful'  and ci.resource_type = 'Course'
    group by 1
    order by 2 
    )
    select
    count(user_id) count_user,
    avg(count_course) avg_course
    from purchased
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
users_course= pd.DataFrame(getUsersCourse())

display(users_course)   # посмотрим, сколько клиентов купили курсы и скотлько в среднем на одного клиента покупок курсов

,count_user,avg_course
0,49006,1.4402522140146105


In [31]:
# Считаем количество клиентов, купивших более одного уникального курса

def getCountUsers():
    query = '''with purchased as
    (select 
    c.user_id user_id,
    ci.resource_id
    from
    final.carts c
    join final.cart_items ci on c.id = ci.cart_id
    where c.state = 'successful'  and ci.resource_type = 'Course'  
    group by 1,2
    order by 1,2
    ),
    user_count as
    (
    select
    user_id,
    resource_id,
    count(resource_id) over (partition by user_id) "count_course"
    from purchased
    order by 3,1
    )
    select 
    count(distinct user_id) count_us,
    count(distinct resource_id) resource_id
    from user_count
    where count_course > 1
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
count_activ_user= pd.DataFrame(getCountUsers())

display(count_activ_user)

,count_us,resource_id
0,12656,126


#### Выводы по первому этапу:
- В анализируемых данных представлены продажи за 2017 и 2018 годы.
- За этот период 49 006 клиентов покупали 126 курсов (из имеющихся 127), из них 12 656 клиентов совершили более одной уникальной покупки курсов.
- Среднее число купленных курсов на одного клиента составило 1,44 уникальных курса.
- Один из курсов никто ни разу не покупал!. Ниже посмотрм, что за курс.   

 #### 2. ОБРАБОТКА ПОЛУЧЕННЫХ ДАННЫХ С ПОМОЩЬЮ PYTHON

In [32]:
# создаем в SQL и выгружаем таблицу с ID клиентов, купивших более одного уникального курса, и с id этих курсов

def getActiveUsersWithCourse():
    query = '''with purchased as
    (select 
    c.user_id user_id,
    ci.resource_id
    from
    final.carts c
    join final.cart_items ci on c.id = ci.cart_id
    where c.state = 'successful'  and ci.resource_type = 'Course'  
    group by 1,2
    order by 1,2
    ),
    user_count as
    (
    select
    user_id,
    resource_id,
    count(resource_id) over (partition by user_id) "count_course"
    from purchased
    order by 3,1
    )
    select 
    user_id as user,
    resource_id as course
    from user_count
    where count_course > 1
    order by 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
df_users_course= pd.DataFrame(getActiveUsersWithCourse())

print(df_users_course.head(), '\n', 'количество строк:', len(df_users_course))

   user  course
0    51     516
1    51    1099
2  6117     356
3  6117     357
4  6117    1125 
 количество строк: 34074


In [33]:
# загрузим библиотеки

from pprint import pprint
from collections import defaultdict
from itertools import combinations
from collections import Counter

In [34]:
# создаем копию исходного ДатаФрейма

df_a0 = df_users_course.copy()

In [35]:
# делаем список уникальных id купленных курсов из первоначальной таблицы)

uniq_course = list(df_a0['course'].unique())
print('количество купленных курсов:',  len(uniq_course))

количество купленных курсов: 126


In [36]:
# находим id курса, который не покупался c помощью сравнений двух множеств (найдем их разность) 

all_course_set = set(course_id['course_id'].tolist())
uniq_course_set = set(uniq_course)
course_off = all_course_set.difference(uniq_course_set)
print('id курса, который никто не покупал:', course_off)

id курса, который никто не покупал: {772}


In [37]:
# группируем в списки все уникальные курсы каждого пользователя

df_course = df_a0.groupby(['user'])['course'].apply(lambda x:list(np.unique(x)))
df_course = df_course.to_frame()
print(df_course.head(), '\n', 'количество строк:', len(df_course))

                 course
user                   
51          [516, 1099]
6117   [356, 357, 1125]
10275       [553, 1147]
10457       [361, 1138]
17166        [356, 357] 
 количество строк: 12656


In [38]:
# для каждого пользователя (user) оздаем все уникальные пары из его списка курсов и помещаем в отдельный столбец (course) ДатаФрейма

paries = df_course['course'].apply(lambda x:list(combinations(x,2)))
paries = paries.to_frame()
print(paries.head(), '\n', 'количество строк:', len(df_course))

                                       course
user                                         
51                              [(516, 1099)]
6117   [(356, 357), (356, 1125), (357, 1125)]
10275                           [(553, 1147)]
10457                           [(361, 1138)]
17166                            [(356, 357)] 
 количество строк: 12656


In [39]:
# переносим столбец со списками пар курсов в отдельный список, смотрим длину списка

paries_lst= paries['course'].tolist()
print(paries_lst[:5], '...', '\n', 'количество строк:', len(paries_lst))

[[(516, 1099)], [(356, 357), (356, 1125), (357, 1125)], [(553, 1147)], [(361, 1138)], [(356, 357)]] ... 
 количество строк: 12656


In [40]:
# преобразовываем список списков пар курсов в единый спиcок пар курсов 

paries_all = sum(paries_lst, [])
print(paries_all[:5], '...', '\n', 'количество строк:', len(paries_all))

[(516, 1099), (356, 357), (356, 1125), (357, 1125), (553, 1147)] ... 
 количество строк: 40017


In [41]:
# и далее переносим в Датафрейм с курасами их парами

df_paries = pd.DataFrame(data=paries_all, columns=['course', 'course2'])
df_paries['paries'] = paries_all
df_paries1 = df_paries.drop(['course','course2'], axis=1)
print(df_paries1.head(), '\n', 'количество строк:', len(df_paries1))

        paries
0  (516, 1099)
1   (356, 357)
2  (356, 1125)
3  (357, 1125)
4  (553, 1147) 
 количество строк: 40017


In [42]:
# создаем таблицу с уникальными парами курсов, проранжированными по частоте их совместных покупок (rank), считаем количество уникальных пар

df_paries2 = df_paries1.value_counts().to_frame(name='rank').reset_index(inplace= False)
print(df_paries2.head(), '\n', 'количество строк:', len(df_paries2))

       paries  rank
0  (551, 566)   797
1  (515, 551)   417
2  (489, 551)   311
3  (523, 551)   304
4  (566, 794)   290 
 количество строк: 3989


#### Выводы по второму этапу:
- Следует обратить внимание на курс с номером 772, поскольку он ни разу не покупался, необходимо разобраться в причинах.
- Среди покупок активных клиентов (купивших более одного курса) встречаются 3989 различных пар курсов.
- Самая популярная пара курсов (551, 566) встречается 797 раз. Возможно, эти курсы очень похожи или дополняют друг друга.

 #### 3. ФОРМИРОВАНИЕ РЕКОМЕНДАТЕЛЬНОЙ ТАБЛИЦЫ С ПОМОЩЬЮ PYTHON

In [43]:
# формируем cписок, содержащий список из пары курсов (курс1, курс2) и рейтингами этих пар(rank)

paries_tuple = tuple()
paries_list = []
for elem in df_paries2.itertuples():
    paries_list.append((elem[1], elem[2]))

paries_list
print(paries_list[:5], '...', '\n', 'количество строк:', len(paries_list))

[((551, 566), 797), ((515, 551), 417), ((489, 551), 311), ((523, 551), 304), ((566, 794), 290)] ... 
 количество строк: 3989


In [44]:
# создаем функцию, собирающую для конкретного курса все его пары с их рейтингами и выдающую двух рекомендуемыых соседей из ТОП2 его соседей, используя рейтинг пар

def recomm_func(course):
    recomm_list = []
    for elem in paries_list:
       if elem[0][0] == course:
          recomm_list.append((elem[1], course, elem[0][1]))
       elif elem[0][1] == course:
           recomm_list.append((elem[1], course, elem[0][0]))
       else:
           continue
    recomm_list.sort(reverse=True)
    cnt=len(recomm_list)
    recomm_list[:2]
    recomm_list = recomm_list[:2]
    recomm_list.append(cnt)
    return recomm_list


In [45]:
# собираем в финальную таблицу купленных курсов c рекомендованными парами к ним, добавим столбец с количеством пар, в которых этот курс встречается.

final_list = []
one_course_list = []
for elem in uniq_course:
    if len(recomm_func(elem)) == 3:
        final_list.append((recomm_func(elem)[0][1], recomm_func(elem)[0][2],recomm_func(elem)[1][2],recomm_func(elem)[2]))
    else:
        final_list.append((recomm_func(elem)[0][1], recomm_func(elem)[0][2],recomm_func(elem)[0][2],recomm_func(elem)[2]))
        one_course_list.append(elem)
    continue

final_df= pd.DataFrame(data=final_list, columns= ['course', 'first_recomm', 'sec_recomm', 'quantity_paries'])


#### Результат третьего этапа: Рекомендательная таблица.

In [46]:
final_df.sort_values(by=['quantity_paries'], ascending=False) 

,course,first_recomm,sec_recomm,quantity_paries
10,551,566,515,117
51,523,551,515,116
23,552,551,523,114
48,515,551,489,114
13,514,551,515,113
...,...,...,...,...
116,1199,523,1115,7
124,833,570,507,7
118,1201,1186,1144,4
121,902,777,742,4


#### Некоторые дополнительные показатели к Рекомендательной таблице

In [47]:
# Проверяем, есть ли курсы, у которых менее двух предложеных рекомендаций
print(one_course_list)
# таких курсов нет

[]


In [48]:
# немного статистической информации о таблице  

final_df=final_df.astype({'course' : object, 'first_recomm' : object,'sec_recomm' : object})
final_df.describe()

,quantity_paries
count,126.000000
mean,63.317460
std,28.667934
min,3.000000
25%,44.250000
50%,64.000000
75%,80.000000
max,117.000000


In [49]:
final_df.describe(include= object)

,course,first_recomm,sec_recomm
count,126,126,126
unique,126,38,51
top,516,551,551
freq,1,33,11


In [50]:
# тут можно посмотреть показатели по интересующему курсу (задав номер курса в переменной arg)

arg = 566
print('показатели курса:', '\n', final_df.loc[final_df['course'] == arg], '\n')
print('количество рекомендаций №1:', final_df.loc[final_df['first_recomm'] == arg]['first_recomm'].count())
print('количество рекомендаций №2:', final_df.loc[final_df['sec_recomm'] == arg]['sec_recomm'].count())
# print(final_df[final_df.loc[:,'first_recomm'].isin([arg])]['first_recomm'].count())


показатели курса: 
    course first_recomm sec_recomm  quantity_paries
15    566          551        794              112 

количество рекомендаций №1: 9
количество рекомендаций №2: 9


#### Выводы по третьему этапу:
- Самым часто встречающимся в парах в другими курсами (с показателем 117 пар) оказался курс 551, он же оказывается и лидером в рекомендациях к остальным (33 первых рекомендаций и 11 вторых)
- Второй курс из самой топовой пары (551, 566) имеет показатели: всего количества сочетаний с другими - 112, рекомендаций №1 - 9 и рекомендаций №2 - 9.
- Средний показатель участия курса в парах с остальными составил 63 пары на курс, при максимальном у ТОП1 - 117.

 #### 4. АНАЛИЗ РЕЗУЛЬТАТОВ ВНЕДРЕНИЯ РЕКОМЕНДАТЕЛЬНОЙ ТАБЛИЦЫ С ПОМОЩЬЮ A/B-ТЕСТА

Для проведения тестирования эффективности внедрения новых рекомендаций все клиенты должны будут случайным образом поделены на контрольную и тестовую группы:    
- тестовой группе будут показаны рекомендации,  
-  а контрольной — нет.

До реализации рекомендаций средняя конверсия в покупку второго курса была 3,2%.   
Ожидаемое после ввода рекомендаций значение показателя конверсии -  4% (относительный рост - 20%).

Сформулируем гитпотезу: 
- HO - Конверсия после внедрения новых рекомендаций увелитcя на 20%
- H1 - Конверсия не вырастет или вырастет меньше ожидаемого

Оопределим минимальный размер выборки для проведения теста при уровне достоверности 95% и статистической мощности 80%.   
Для расчета воспользуемся калькулятором: https://www.evanmiller.org/ab-testing/sample-size.html  
##### ВАЖНО УЧИТЫВАТЬ!:
 - При заданных ожидамых параметрах необходимый размер выборки для каждой группы должен будет составить 7900 клиентов.  
 - При среднемесячном показателе количествва покупок в прошлом порядка 2000 / мес. (49000 / 24) для проведения эксперимента потребуется ОКОЛО 8 МЕСЯЦЕВ, (а не три недели)!   



##### Оценка результатов проведенного эксперимента
Численное выражение полученных результатов таково:   
В контрольной группе оказалось 8732 клиента, оформивших заказ, из них 293 купили больше одного курса.
В тестовой — 8847 клиентов, из них 347 купили больше одного курса.

1. Расчитаем p-value, воспользовавшись калькулятором   https://abtestguide.com/calc/ .   Получим p-value  равным 0.0224 (или 2,2%)

##### ВЫВОД ПО ИТОГАМ A/B ТЕСТА::
 - Величина p-value 2,2% оказалась ниже заданного нами уровня достоверности 5%. Значит А/В тест подтвердил гипотезу о 20% росте конверсии.
 - Вариант с рекомендациями показал статистически значимо лучший результат, нужно реализовывать его для всех клиентов. 



